In [ ]:
```xml
<VSCode.Cell id="#VSC-edc8b810" language="python">
import sys
sys.path.insert(0, '..')
sys.path.insert(0, '../src')


import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from src.data.preprocess import extend_market_data

# Define ticker symbols for corn and ethanol futures
corn_ticker = "ZC=F"   # Corn Futures (CBOT)
ethanol_ticker = "ZS=F"  # Ethanol Futures (NYMEX)
corn = yf.Ticker(corn_ticker)
ethanol = yf.Ticker(ethanol_ticker)
corn_data = corn.history(start ="2014-01-01", end ="2024-12-31")
ethanol_data = ethanol.history(start ="2014-01-01", end ="2024-12-31")

corn_data = extend_market_data(corn_data)
ethanol_data = extend_market_data(ethanol_data)

features = list(corn_data.columns)
features.remove('expiry')
corn_data = corn_data[features]

features = list(ethanol_data.columns)
features.remove('expiry')
ethanol_data = ethanol_data[features]

if merged.index.name != 'Date':
    merged = merged.set_index('Date')
merged = merged.drop(columns = ["next_day_increment", "next_3day_increment", "next_7day_increment"])

corn_data["Log_Return_Shift"] = corn_data["Log_Return"].shift(-1)

from datetime import datetime
# start_date2231 = datetime.fromisoformat("2014-01-02")
import shelve
with shelve.open('feat_dict.db') as features:

    # Convert your features dict to a DataFrame
    features_df = pd.DataFrame.from_dict(features, orient='index')
    features_df.index = pd.to_datetime(features_df.index)
    def create_shifted_features(features_df, lags):
        shifted_dfs = []
        for lag in lags:
            shifted = features_df.shift(lag)
            shifted.columns = [f"{col}_lag_{lag}" for col in features_df.columns]
            shifted_dfs.append(shifted)
    
        # Concatenate all lagged features side-by-side
        return pd.concat(shifted_dfs, axis=1)

    lags = [2**i for i in range(1)]
    lagged_features = create_shifted_features(features_df, lags)
    # print(lagged_features.columns)
    features_df = features_df.join(lagged_features, how = 'inner')
    # Combine with your existing df
    corn_data_with_weather = corn_data.join(features_df, how='left')
    corn_data_with_weather = corn_data_with_weather.dropna()
merged = merged.drop(columns=corn_data_with_weather.columns.intersection(merged.columns))
merged = merged.dropna()
corn_data_with_weather = corn_data_with_weather.dropna()
corn_data_with_weather = corn_data_with_weather.join(merged, how='inner')
# from sklearn.preprocessing import PolynomialFeatures

# poly = PolynomialFeatures(degree=2, include_bias=False)
# poly_array = poly.fit_transform(corn_data_with_weather)
# poly_feature_names = poly.get_feature_names_out(corn_data_with_weather.columns)
# corn_data_with_weather = pd.DataFrame(poly_array, columns=poly_feature_names, index=corn_data_with_weather.index)

corn_data_train = corn_data[:datetime.fromisoformat("2023-12-31")]
corn_data_test = corn_data[datetime.fromisoformat("2023-12-31"):]
corn_data_with_weather_train = corn_data_with_weather[:datetime.fromisoformat("2023-12-31")]
corn_data_with_weather_test = corn_data_with_weather[datetime.fromisoformat("2023-12-31"):]

</VSCode.Cell>
<VSCode.Cell id="#VSC-d6dcc87c" language="markdown">
Linear Regression with no weather data
</VSCode.Cell>
<VSCode.Cell id="#VSC-009b46fb" language="python">
from sklearn.linear_model import LinearRegression
import numpy as np
import xgboost as xgb
from sklearn.datasets import make_regression
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV

corn_data2 = corn_data_train.dropna()

lin = LinearRegression()
features = list(corn_data.columns)
features.remove('Log_Return_Shift')
X = corn_data2[list(features)]
y = corn_data2["Log_Return_Shift"]
tss = TimeSeriesSplit(n_splits = 10)
r2 = 0
for i, (train_index, test_index) in enumerate(tss.split(X)):
    X_train = X.iloc[train_index]
    X_test = X.iloc[test_index]
    y_train = y.iloc[train_index]
    y_test = y.iloc[test_index]
    lin.fit(X_train,y_train)
    predlin = lin.predict(X_test)
    r2 = r2_score(y_test, predlin)
    print(f"R² Score Linear: {r2:.4f}")
</VSCode.Cell>
<VSCode.Cell id="#VSC-5a9b500a" language="markdown">
Linear regression with weather data
</VSCode.Cell>
<VSCode.Cell id="#VSC-bb1f78ea" language="python">
from sklearn.linear_model import LinearRegression

features = list(corn_data_with_weather.columns)
features.remove('Log_Return_Shift')

corn_data2 = corn_data_with_weather_train.dropna()

lin_weather = LinearRegression()
# features = ["average_temperature_distribution_weighted_kurtosis"]
X = corn_data_with_weather_train[list(features)]
y = corn_data_with_weather_train["Log_Return_Shift"]
tss = TimeSeriesSplit(n_splits = 10)
r2 = 0
for i, (train_index, test_index) in enumerate(tss.split(X)):
    X_train = X.iloc[train_index]
    X_test = X.iloc[test_index]
    y_train = y.iloc[train_index]
    y_test = y.iloc[test_index]
    lin_weather.fit(X_train,y_train)
    predlin = lin_weather.predict(X_test)
    r2 = r2_score(y_test, predlin)
    print(f"R² Score Linear: {r2:.4f}")
</VSCode.Cell>
<VSCode.Cell id="#VSC-6c583715" language="python">
def obj(params):

    if 'n_estimators' in params.keys():
        params['n_estimators'] = int(params['n_estimators'])
    if 'max_depth' in params.keys():
        params['max_depth'] = int(params['max_depth'])
    
    
    features = list(corn_data.columns)
    features.remove('Log_Return_Shift')
    
    corn_data2 = corn_data_train.dropna()
    X = corn_data2[features]
    y = corn_data2["Log_Return_Shift"]
    tss = TimeSeriesSplit(n_splits = 5)
    r2 = 0
    for i, (train_index, test_index) in enumerate(tss.split(X)):
        
        X_train = X.iloc[train_index]
        X_test = X.iloc[test_index]
        y_train = y.iloc[train_index]
        y_test = y.iloc[test_index]
        
        dtrain = xgb.DMatrix(X_train, label=y_train)
        dtest = xgb.DMatrix(X_test, label=y_test)
        
        xgb_model = xgb.XGBRegressor(**params)
        xgb_model.fit(X_train, y_train)
        y_pred = xgb_model.predict(X_test)
        sc = r2_score(y_test, y_pred)
        r2 += r2_score(y_test, y_pred)
    return -r2

def obj_with_weather(params):

    if 'n_estimators' in params.keys():
        params['n_estimators'] = int(params['n_estimators'])
    if 'max_depth' in params.keys():
        params['max_depth'] = int(params['max_depth'])
    
    
    features = list(corn_data_with_weather_train.columns)
    features.remove('Log_Return_Shift')

    corn_data2 = corn_data_with_weather_train.dropna()
    X = corn_data2[features]
    y = corn_data2["Log_Return_Shift"]
    tss = TimeSeriesSplit(n_splits = 5)
    r2 = 0
    for i, (train_index, test_index) in enumerate(tss.split(X)):
        
        X_train = X.iloc[train_index]
        X_test = X.iloc[test_index]
        y_train = y.iloc[train_index]
        y_test = y.iloc[test_index]
        
        dtrain = xgb.DMatrix(X_train, label=y_train)
        dtest = xgb.DMatrix(X_test, label=y_test)
        
        xgb_model = xgb.XGBRegressor(**params)
        xgb_model.fit(X_train, y_train)
        y_pred = xgb_model.predict(X_test)
        sc = r2_score(y_test, y_pred)
        r2 += r2_score(y_test, y_pred)
    return -r2
</VSCode.Cell>
<VSCode.Cell id="#VSC-cb18665b" language="python">
from hyperopt import fmin, tpe, hp
# Define the hyperparameter space
paramspace = {
    'objective': 'reg:squarederror',  # Regression
    'eval_metric': 'rmse',
    'n_estimators': hp.quniform('n_estimators', 10,200,10),
    'max_depth': hp.quniform('max_depth', 2, 8, 1),
    'learning_rate': hp.loguniform('learning_rate', -5, -1),
    'min_child_weight': hp.quniform('min_child_weight', 1, 100, 9),
    'gamma': hp.quniform('gamma', 0, 5, 1),
    'subsample': hp.uniform('subsample', .5, 1)
}

</VSCode.Cell>
<VSCode.Cell id="#VSC-0751a2f4" language="markdown">
xgboost with no weather data
</VSCode.Cell>
<VSCode.Cell id="#VSC-6dc1ed72" language="python">
best_params_no_weather = fmin(obj, paramspace, algo=tpe.suggest, max_evals=1000)
</VSCode.Cell>
<VSCode.Cell id="#VSC-edf2dde1" language="markdown">
xgboost with weather data
</VSCode.Cell>
<VSCode.Cell id="#VSC-533c3045" language="python">
best_params_with_weather = fmin(obj_with_weather, paramspace, algo=tpe.suggest, max_evals=1000)
</VSCode.Cell>
<VSCode.Cell id="#VSC-86a6f47b" language="markdown">
XGBoost with no weather data
</VSCode.Cell>
<VSCode.Cell id="#VSC-ae93e148" language="python">
obj(best_params_no_weather)
</VSCode.Cell>
<VSCode.Cell id="#VSC-94d373dd" language="markdown">
XGBoost with weather data
</VSCode.Cell>
<VSCode.Cell id="#VSC-5e224a67" language="python">
obj_with_weather(best_params_with_weather)
</VSCode.Cell>
<VSCode.Cell id="#VSC-87fb632a" language="markdown">
Neural network with no weather
</VSCode.Cell>
<VSCode.Cell id="#VSC-7d97f71a" language="python">
combined_df = corn_data.dropna()

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

# Fit the scaler on the training data and transform it.
scaled = scaler.fit_transform(combined_df)

# Convert the NumPy arrays back into DataFrames.
scaled_df = pd.DataFrame(scaled, index=combined_df.index, columns=combined_df.columns)

import torch
import torch.nn as nn
import torch.optim as optim

# Set device for PyTorch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Prepare target and predictors as NumPy arrays.
# combined_df should contain the target 'Log_Return_Shift' and predictors.
y = scaled_df['Log_Return_Shift'].values
X = scaled_df.drop(columns=['Log_Return_Shift']).values

# Define a simple feed-forward neural network model.
class FeedForwardNN(nn.Module):
    def __init__(self, input_dim):
        super(FeedForwardNN, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )
    
    def forward(self, x):
        return self.model(x)

# Training function with early stopping.
def train_model(model, optimizer, criterion, X_train, y_train, X_val, y_val,
                num_epochs=100, batch_size=32, patience=10):
    model.train()
    n_train = X_train.shape[0]
    best_val_loss = np.inf
    epochs_no_improve = 0
    best_model_state = None

    # Convert all training and validation data to tensors.
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
    y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1).to(device)
    X_val_tensor = torch.tensor(X_val, dtype=torch.float32).to(device)
    y_val_tensor = torch.tensor(y_val, dtype=torch.float32).view(-1, 1).to(device)
    
    for epoch in range(num_epochs):
        model.train()
        permutation = torch.randperm(n_train)
        epoch_loss = 0.0

        # Mini-batch training.
        for i in range(0, n_train, batch_size):
            optimizer.zero_grad()
            indices = permutation[i:i+batch_size]
            batch_x = X_train_tensor[indices]
            batch_y = y_train_tensor[indices]
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item() * batch_x.size(0)
        
        epoch_loss /= n_train

        # Evaluate on validation data.
        model.eval()
        with torch.no_grad():
            val_outputs = model(X_val_tensor)
            val_loss = criterion(val_outputs, y_val_tensor).item()
        
        # Early stopping check.
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model_state = model.state_dict()
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                break

    # Restore best model state.
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
    
    return model

# Set up TimeSeriesSplit for cross-validation.
tscv = TimeSeriesSplit(n_splits=5)
nn_errors = []  # To store (RMSE, R^2) for each fold.

for fold, (train_index, test_index) in enumerate(tscv.split(combined_df)):
    # Get fold data.
    train_data = combined_df.iloc[train_index]
    test_data = combined_df.iloc[test_index]
    
    X_train = train_data.drop(columns=['Log_Return_Shift']).values
    y_train = train_data['Log_Return_Shift'].values
    X_test = test_data.drop(columns=['Log_Return_Shift']).values
    y_test = test_data['Log_Return_Shift'].values
    
    # Further split training data to have a validation set (e.g., 80/20 split).
    split_idx = int(0.8 * X_train.shape[0])
    X_train_part, X_val = X_train[:split_idx], X_train[split_idx:]
    y_train_part, y_val = y_train[:split_idx], y_train[split_idx:]
    
    input_dim = X_train.shape[1]
    model = FeedForwardNN(input_dim).to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.MSELoss()
    
    # Train with early stopping.
    model = train_model(model, optimizer, criterion,
                        X_train_part, y_train_part, X_val, y_val,
                        num_epochs=100, batch_size=32, patience=10)
    
    # Evaluate on the test set.
    model.eval()
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
    with torch.no_grad():
        test_pred = model(X_test_tensor).cpu().numpy().flatten()
    
    nn_rmse = np.sqrt(mean_squared_error(y_test, test_pred))
    nn_r2 = r2_score(y_test, test_pred)
    nn_errors.append((nn_rmse, nn_r2))
    
    print(f"Fold {fold+1}: NN RMSE: {nn_rmse:.4f}, NN R^2: {nn_r2:.4f}")

# Compute average metrics across folds.
nn_errors_arr = np.array(nn_errors)
avg_nn_rmse, avg_nn_r2 = nn_errors_arr.mean(axis=0)
print("\nAverage Neural Network Metrics (PyTorch):")
print(f"  Average RMSE: {avg_nn_rmse:.4f}")
print(f"  Average R^2: {avg_nn_r2:.4f}")

</VSCode.Cell>
<VSCode.Cell id="#VSC-167a753f" language="markdown">
Neural network with weather
</VSCode.Cell>
<VSCode.Cell id="#VSC-9daab0c0" language="python">
combined_df = corn_data_with_weather.dropna()

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

# Fit the scaler on the training data and transform it.
scaled = scaler.fit_transform(combined_df)

# Convert the NumPy arrays back into DataFrames.
scaled_df = pd.DataFrame(scaled, index=combined_df.index, columns=combined_df.columns)

import torch
import torch.nn as nn
import torch.optim as optim

# Set device for PyTorch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Prepare target and predictors as NumPy arrays.
# combined_df should contain the target 'Log_Return_Shift' and predictors.
y = scaled_df['Log_Return_Shift'].values
X = scaled_df.drop(columns=['Log_Return_Shift']).values

# Define a simple feed-forward neural network model.
class FeedForwardNN(nn.Module):
    def __init__(self, input_dim):
        super(FeedForwardNN, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )
    
    def forward(self, x):
        return self.model(x)

# Training function with early stopping.
def train_model(model, optimizer, criterion, X_train, y_train, X_val, y_val,
                num_epochs=100, batch_size=32, patience=10):
    model.train()
    n_train = X_train.shape[0]
    best_val_loss = np.inf
    epochs_no_improve = 0
    best_model_state = None

    # Convert all training and validation data to tensors.
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
    y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1).to(device)
    X_val_tensor = torch.tensor(X_val, dtype=torch.float32).to(device)
    y_val_tensor = torch.tensor(y_val, dtype=torch.float32).view(-1, 1).to(device)
    
    for epoch in range(num_epochs):
        model.train()
        permutation = torch.randperm(n_train)
        epoch_loss = 0.0

        # Mini-batch training.
        for i in range(0, n_train, batch_size):
            optimizer.zero_grad()
            indices = permutation[i:i+batch_size]
            batch_x = X_train_tensor[indices]
            batch_y = y_train_tensor[indices]
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item() * batch_x.size(0)
        
        epoch_loss /= n_train

        # Evaluate on validation data.
        model.eval()
        with torch.no_grad():
            val_outputs = model(X_val_tensor)
            val_loss = criterion(val_outputs, y_val_tensor).item()
        
        # Early stopping check.
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model_state = model.state_dict()
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                break

    # Restore best model state.
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
    
    return model

# Set up TimeSeriesSplit for cross-validation.
tscv = TimeSeriesSplit(n_splits=5)
nn_errors = []  # To store (RMSE, R^2) for each fold.

for fold, (train_index, test_index) in enumerate(tscv.split(combined_df)):
    # Get fold data.
    train_data = combined_df.iloc[train_index]
    test_data = combined_df.iloc[test_index]
    
    X_train = train_data.drop(columns=['Log_Return_Shift']).values
    y_train = train_data['Log_Return_Shift'].values
    X_test = test_data.drop(columns=['Log_Return_Shift']).values
    y_test = test_data['Log_Return_Shift'].values
    
    # Further split training data to have a validation set (e.g., 80/20 split).
    split_idx = int(0.8 * X_train.shape[0])
    X_train_part, X_val = X_train[:split_idx], X_train[split_idx:]
    y_train_part, y_val = y_train[:split_idx], y_train[split_idx:]
    
    input_dim = X_train.shape[1]
    model = FeedForwardNN(input_dim).to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.MSELoss()
    
    # Train with early stopping.
    model = train_model(model, optimizer, criterion,
                        X_train_part, y_train_part, X_val, y_val,
                        num_epochs=100, batch_size=32, patience=10)
    
    # Evaluate on the test set.
    model.eval()
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
    with torch.no_grad():
        test_pred = model(X_test_tensor).cpu().numpy().flatten()
    
    nn_rmse = np.sqrt(mean_squared_error(y_test, test_pred))
    nn_r2 = r2_score(y_test, test_pred)
    nn_errors.append((nn_rmse, nn_r2))
    
    print(f"Fold {fold+1}: NN RMSE: {nn_rmse:.4f}, NN R^2: {nn_r2:.4f}")

# Compute average metrics across folds.
nn_errors_arr = np.array(nn_errors)
avg_nn_rmse, avg_nn_r2 = nn_errors_arr.mean(axis=0)
print("\nAverage Neural Network Metrics (PyTorch):")
print(f"  Average RMSE: {avg_nn_rmse:.4f}")
print(f"  Average R^2: {avg_nn_r2:.4f}")

</VSCode.Cell>
<VSCode.Cell id="#VSC-a95b3e71" language="markdown">
LSTM Models
</VSCode.Cell>
<VSCode.Cell id="#VSC-ed3256df" language="python">
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score

device = "cpu"
print(f"Using {device} device")
</VSCode.Cell>
<VSCode.Cell id="#VSC-37737af9" language="markdown">
LSTM with no weather
</VSCode.Cell>
<VSCode.Cell id="#VSC-12873099" language="python">
data = corn_data.dropna()

def create_sequences(data, seq_length, features):
    scaler = MinMaxScaler(feature_range=(-1, 1))
    data_scaled = scaler.fit_transform(data)
    
    features = list(data.columns)
    features.remove("Log_Return_Shift")
    
    feature_frame = data[features]
    target_series = data["Log_Return_Shift"]
    sequences = []
    targets = []
    for i in range(len(data) - seq_length):
        sequences.append(feature_frame.iloc[i:i+seq_length])
        targets.append(target_series.iloc[i+seq_length])
    return np.array(sequences), np.array(targets)



SEQ_LENGTH = 60
X, y = create_sequences(data, SEQ_LENGTH, features)

# Convert to PyTorch tensors
X_train, y_train = torch.tensor(X[:-500], dtype=torch.float32).to(device), torch.tensor(y[:-500], dtype=torch.float32).to(device)
X_test, y_test = torch.tensor(X[-500:], dtype=torch.float32).to(device), torch.tensor(y[-500:], dtype=torch.float32).to(device)

# print(X_train.shape[2])
# Reshape for LSTM (batch_size, seq_length, num_features)
X_train = X_train.view(-1, SEQ_LENGTH, X_train.shape[2])
X_test = X_test.view(-1, SEQ_LENGTH, X_train.shape[2])

# Create DataLoader
BATCH_SIZE = 32
train_loader = torch.utils.data.DataLoader(TensorDataset(X_train, y_train), batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(TensorDataset(X_test, y_test), batch_size=BATCH_SIZE, shuffle=False)
</VSCode.Cell>
<VSCode.Cell id="#VSC-f4f5f184" language="python">
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim=64, num_layers=2, output_dim=1):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        # print(lstm_out)
        out = self.fc(lstm_out[:, -1, :])  # Take last output from LSTM
        # print(out)
        return out

# Initialize Model
model = LSTMModel(input_dim = X_train.shape[2]).to(device)

</VSCode.Cell>
<VSCode.Cell id="#VSC-864207dc" language="python">
# Define Loss and Optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training Loop
EPOCHS = 100
for epoch in range(EPOCHS):
    model.train()
    epoch_loss = 0
    
    for batch_x, batch_y in train_loader:
        # if(torch.isnan(batch_x).any() or torch.isinf(batch_x).any() or torch.isnan(batch_y).any() or torch.isinf(batch_y).any()):
        #     print("hi")
        # print(torch.isnan(batch_x).any(), torch.isinf(batch_x).any())
        # print(torch.isnan(batch_y).any(), torch.isinf(batch_y).any())
        optimizer.zero_grad()
        y_pred = model(batch_x)
        # print(y_pred)
        loss = criterion(y_pred.squeeze(), batch_y)
        # print(y_pred.squeeze())
        # print(batch_y)
        # print(loss)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{EPOCHS}], Loss: {epoch_loss/len(train_loader):.6f}')
</VSCode.Cell>
<VSCode.Cell id="#VSC-d35e8678" language="python">
model.eval()
with torch.no_grad():
    y_pred_test = model(X_test).squeeze().cpu().numpy()

# Inverse transform predictions
# y_pred_test_rescaled = scaler.inverse_transform(y_pred_test.reshape(-1, 1))
# y_test_rescaled = scaler.inverse_transform(y_test.numpy().reshape(-1, 1))

y_pred_test_rescaled = y_pred_test.reshape(-1, 1)
y_test_rescaled = y_test.cpu().numpy().reshape(-1, 1)

# Plot results

plt.figure(figsize=(12, 6))
plt.plot(data.index[-500:], y_test_rescaled, label='Actual')
plt.plot(data.index[-500:], y_pred_test_rescaled, label='Predicted')
plt.legend()
plt.title("LSTM Model Predictions on testing set")
plt.show()

print(f"r2 score {r2_score(y_test_rescaled, y_pred_test_rescaled):.4f}")
</VSCode.Cell>
<VSCode.Cell id="#VSC-afad3d49" language="markdown">
LSTM with weather
</VSCode.Cell>
<VSCode.Cell id="#VSC-e6bb5c2e" language="python">
data = corn_data_with_weather.dropna()

def create_sequences(data, seq_length, features):
    scaler = MinMaxScaler(feature_range=(-1, 1))
    data_scaled = scaler.fit_transform(data)
    
    features = list(data.columns)
    features.remove("Log_Return_Shift")
    
    feature_frame = data[features]
    target_series = data["Log_Return_Shift"]
    sequences = []
    targets = []
    for i in range(len(data) - seq_length):
        sequences.append(feature_frame.iloc[i:i+seq_length])
        targets.append(target_series.iloc[i+seq_length])
    return np.array(sequences), np.array(targets)



SEQ_LENGTH = 60
X, y = create_sequences(data, SEQ_LENGTH, features)

# Convert to PyTorch tensors
X_train, y_train = torch.tensor(X[:-500], dtype=torch.float32).to(device), torch.tensor(y[:-500], dtype=torch.float32).to(device)
X_test, y_test = torch.tensor(X[-500:], dtype=torch.float32).to(device), torch.tensor(y[-500:], dtype=torch.float32).to(device)

# print(X_train.shape[2])
# Reshape for LSTM (batch_size, seq_length, num_features)
X_train = X_train.view(-1, SEQ_LENGTH, X_train.shape[2])
X_test = X_test.view(-1, SEQ_LENGTH, X_train.shape[2])

# Create DataLoader
BATCH_SIZE = 32
train_loader = torch.utils.data.DataLoader(TensorDataset(X_train, y_train), batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(TensorDataset(X_test, y_test), batch_size=BATCH_SIZE, shuffle=False)
</VSCode.Cell>
<VSCode.Cell id="#VSC-4e67a22b" language="python">
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim=64, num_layers=2, output_dim=1):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        # print(lstm_out)
        out = self.fc(lstm_out[:, -1, :])  # Take last output from LSTM
        # print(out)
        return out

# Initialize Model
model = LSTMModel(input_dim = X_train.shape[2]).to(device)

</VSCode.Cell>
<VSCode.Cell id="#VSC-fb842d66" language="python">
# Define Loss and Optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training Loop
EPOCHS = 100
for epoch in range(EPOCHS):
    model.train()
    epoch_loss = 0
    
    for batch_x, batch_y in train_loader:
        # if(torch.isnan(batch_x).any() or torch.isinf(batch_x).any() or torch.isnan(batch_y).any() or torch.isinf(batch_y).any()):
        #     print("hi")
        # print(torch.isnan(batch_x).any(), torch.isinf(batch_x).any())
        # print(torch.isnan(batch_y).any(), torch.isinf(batch_y).any())
        optimizer.zero_grad()
        y_pred = model(batch_x)
        # print(y_pred)
        loss = criterion(y_pred.squeeze(), batch_y)
        # print(y_pred.squeeze())
        # print(batch_y)
        # print(loss)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{EPOCHS}], Loss: {epoch_loss/len(train_loader):.6f}')
</VSCode.Cell>
<VSCode.Cell id="#VSC-cf177f81" language="python">
model.eval()
with torch.no_grad():
    y_pred_test = model(X_test).squeeze().cpu().numpy()

# Inverse transform predictions
# y_pred_test_rescaled = scaler.inverse_transform(y_pred_test.reshape(-1, 1))
# y_test_rescaled = scaler.inverse_transform(y_test.numpy().reshape(-1, 1))

y_pred_test_rescaled = y_pred_test.reshape(-1, 1)
y_test_rescaled = y_test.cpu().numpy().reshape(-1, 1)

# Plot results

plt.figure(figsize=(12, 6))
plt.plot(data.index[-500:], y_test_rescaled, label='Actual')
plt.plot(data.index[-500:], y_pred_test_rescaled, label='Predicted')
plt.legend()
plt.title("LSTM Model Predictions on testing set")
plt.show()

print(f"r2 score {r2_score(y_test_rescaled, y_pred_test_rescaled):.4f}")
</VSCode.Cell>
<VSCode.Cell id="#VSC-9d54df76" language="markdown">
ARIMA with no weather
</VSCode.Cell>
<VSCode.Cell id="#VSC-829afeaa" language="python">
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller

df_sugar = corn_data.dropna()

split_index = int(len(df_sugar)*0.8)
df_train = df_sugar[:split_index]['Log_Return_Shift']
df_test = df_sugar[split_index:]['Log_Return_Shift']

# Perform ADF test to check for stationarity
result = adfuller(df_sugar['Log_Return_Shift'])
print(f"ADF Statistic: {result[0]}")
print(f"p-value: {result[1]}")

</VSCode.Cell>
<VSCode.Cell id="#VSC-908d0f98" language="python">
# Fit a SARIMA model with seasonal order (p, d, q, s)
model = SARIMAX(df_train, order=(30, 1, 1), seasonal_order=(0, 1, 0, 12))  # (p, d, q, s)
model_fit = model.fit()

# Summary of the ARIMA model
print(model_fit.summary())

</VSCode.Cell>
<VSCode.Cell id="#VSC-9f7e3368" language="markdown">
ARIMA with weather
</VSCode.Cell>
<VSCode.Cell id="#VSC-cfa506ea" language="python">
# Make predictions (forecast) on the same dataset (for simplicity, we're using the same data for both)
forecast_steps = 40 # Number of periods you want to predict
forecast = model_fit.forecast(steps=forecast_steps)

# Visualizing the forecast
plt.figure(figsize=(10, 6))
plt.plot(df_test[:forecast_steps], label='Historical Data')
plt.plot(df_test.index[:forecast_steps], forecast, label='Forecast', color='red')
plt.title('ARIMA Forecast')
plt.xticks(rotation=45)
plt.legend()
plt.show()
</VSCode.Cell>
<VSCode.Cell id="#VSC-73d90285" language="python">
from sklearn.metrics import root_mean_squared_error as rmse
from sklearn.metrics import r2_score

days = [5,10,15,20,25,30]
for day in days:
    print(f'Days = {day}')
    print('Standard deviaiton of test data:', df_test[:day].std())
    print('Root MSE:', rmse(df_test[:day],forecast[:day]))
    print('R^2 score:', r2_score(df_test[:day],forecast[:day]))
    print()
</VSCode.Cell>
<VSCode.Cell id="#VSC-1c5180ef" language="python">
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller

df_sugar = corn_data_with_weather

split_index = int(len(df_sugar)*0.8)
df_train = df_sugar[:split_index]['Log_Return_Shift']
df_test = df_sugar[split_index:]['Log_Return_Shift']

# Perform ADF test to check for stationarity
result = adfuller(df_sugar['Log_Return_Shift'])
print(f"ADF Statistic: {result[0]}")
print(f"p-value: {result[1]}")

</VSCode.Cell>
<VSCode.Cell id="#VSC-e06f60ac" language="python">
# Fit a SARIMA model with seasonal order (p, d, q, s)
model = SARIMAX(df_train, order=(30, 1, 1), seasonal_order=(0, 1, 0, 12))  # (p, d, q, s)
model_fit = model.fit()

# Summary of the ARIMA model
print(model_fit.summary())

</VSCode.Cell>
<VSCode.Cell id="#VSC-4611a58d" language="python">
# Make predictions (forecast) on the same dataset (for simplicity, we're using the same data for both)
forecast_steps = 40 # Number of periods you want to predict
forecast = model_fit.forecast(steps=forecast_steps)

# Visualizing the forecast
plt.figure(figsize=(10, 6))
plt.plot(df_test[:forecast_steps], label='Historical Data')
plt.plot(df_test.index[:forecast_steps], forecast, label='Forecast', color='red')
plt.title('ARIMA Forecast')
plt.xticks(rotation=45)
plt.legend()
plt.show()
</VSCode.Cell>
<VSCode.Cell id="#VSC-21a162d1" language="python">
from sklearn.metrics import root_mean_squared_error as rmse
from sklearn.metrics import r2_score

days = [5,10,15,20,25,30]
for day in days:
    print(f'Days = {day}')
    print('Standard deviaiton of test data:', df_test[:day].std())
    print('Root MSE:', rmse(df_test[:day],forecast[:day]))
    print('R^2 score:', r2_score(df_test[:day],forecast[:day]))
    print()
</VSCode.Cell>
```